In [2]:
import unicodedata

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [31]:
def quitar_tildes(texto):
    # Normalizar el texto para descomponer los caracteres acentuados
    normalizado = unicodedata.normalize('NFD', texto)
    # Filtrar los caracteres diacríticos combinados
    texto_limpio = ''.join(c for c in normalizado if not unicodedata.combining(c))
    return texto_limpio

In [32]:
df = pd.read_excel("../data/raw/Reporte SSMOdigital Convenios.xlsx")
df["NomRevisor"] = df["NomRevisor"].str.split().str.join(" ")
df["NomRevisor"] = df["NomRevisor"].apply(quitar_tildes)

## Identificar la procedencia de cada uno de los movimientos de un convenio

En este apartado se quiere identificar la unidad en donde se ha realizado un movimiento a cada
convenio.


En este apartado se calcula la cantidad de tiempo utilizado entre cada uno de los movimiento de
un convenio.


In [33]:
# Deja solamente los convenios y los ordena por su numero y fecha de accion
df_convenios = df.query("Categoria == 'Convenio con Entidades Públicas'").copy()
df_convenios = df_convenios.sort_values(["NumInterno", "FechaHistorico"])

# Calcula el tiempo entre 2 acciones distintas para cada convenio
df_convenios["tiempo_utilizado"] = df_convenios.groupby("NumInterno")["FechaHistorico"].diff()

# Calcula el tiempo total utilizado para cada convenio
tiempo_total_por_convenio = df_convenios.groupby("NumInterno")["FechaHistorico"].apply(
    lambda x: x.max() - x.min()
)

En este apartado se identifica cada una de las personas en el SSMO, y se formatea su nombre para
unirlo a los movimientos de los convenios.


In [34]:
df_personas = pd.read_excel(
    "../data/raw/Plano DSSMO dic 2023.xlsx",
    usecols=[
        "Nombre Funcionario",
        "Código Unidad",
        "Descripción Unidad",
        "Código Unidad 2",
        "Descripción Unidad 2",
    ],
).drop_duplicates()

Los nombres de las personas estan escritos en distintos formatos en las bases de datos.

- En los convenios estan escritos como: PRIMER NOMBRE - PRIMER APELLIDO - SEGUNDO APELLIDO.
- En el registro de personas estan en el formato: PRIMER APELLIDO - SEGUNDO APELLIDO - PRIMER NOMBRE - SEGUNDO NOMBRE.

Por lo tanto, es necesario llegar a un formato similar.


In [35]:
# Separa los nombres de los funcionarios para llevarlos al formato
nombres_separados = df_personas["Nombre Funcionario"].apply(quitar_tildes)
nombres_separados = nombres_separados.str.split()
df_personas["nombre_formateado"] = (
    nombres_separados.str[2] + " " + nombres_separados.str[0] + " " + nombres_separados.str[1]
)
# Filtra columnas necesarias
# df_personas = df_personas[["nombre_formateado", "Nombre Funcionario", "Descripción Unidad"]]

In [36]:
# Une ambas bases de datos segun el nombre del funcionario
convenios_con_procedencia = df_convenios.merge(
    df_personas, how="left", left_on="NomRevisor", right_on="nombre_formateado"
)

In [37]:
convenios_con_procedencia.query("nombre_formateado.isna()")["NomRevisor"].value_counts()

NomRevisor
LUZ MARIA BRITO GONZALEZ               28
JUAN PABLO BRAVO QUINTANA              27
PABLO IGNACIO ECHIBURU PEREZ           21
ROSE MARIE FRESARD MEIER               18
FRANCO ROBERTO LOYOLA DIAZ             14
ROBERTO CARLOS VERA ROJAS              13
JUAN PABLO TAGLE ROJAS                 11
SSMODIGITAL SSMODIGITAL SSMODIGITAL    11
ANDREA PERALTA DE LA ROSA               8
BELGICA SAN CRISTOBAL ROSALES           8
MARIA PAZ LAVIN PALMA                   8
CAROLA PAULINA LOPEZ HIDALGO            8
DANIELA ANDREA RIVAS OLIVARES           7
CRISTOBAL MARCELO MOYA GALVEZ           5
JAVIERA MOLINA CAMPOS                   2
MARIA DE LOS ANGELES ZUNIGA HIDALGO     1
Name: count, dtype: int64

In [38]:
df_personas[df_personas["Nombre Funcionario"].str.contains("SAN CRISTOBAL")]

,Nombre Funcionario,Código Unidad,Descripción Unidad,Código Unidad 2,Descripción Unidad 2,nombre_formateado
178,SAN CRISTOBAL ROSALES BELGICA,4319,UNIDAD GESTION PROGRAMA SALUD DSSMO,4037,DEPARTAMENTO GESTION APS,ROSALES SAN CRISTOBAL


In [23]:
df_personas["Nombre Funcionario"].str.split().str.len().value_counts()

Nombre Funcionario
4    403
3     40
5     28
6      9
7      1
Name: count, dtype: int64

In [24]:
df_personas[df_personas["Nombre Funcionario"].str.split().str.len() == 3]

,Nombre Funcionario,Código Unidad,Descripción Unidad,Código Unidad 2,Descripción Unidad 2,nombre_formateado
0,VARGAS PEYREBLANQUE ALBERTO,4069,DIRECCION,4069,DIRECCION,ALBERTO VARGAS PEYREBLANQUE
10,ORTEGA LIEN GABRIELA,1905,CENTRO MEDICO DE FUNCIONARIOS,4042,DEPTO CALIDAD DE VIDA LABORAL DSSMO,GABRIELA ORTEGA LIEN
11,LEPPE MADRID LILIAN,1964,DEPTO DE GESTION DE LAS PERSONAS,4038,SUBDIR GESTION Y DESARROLL PERSONAS SSMO,LILIAN LEPPE MADRID
22,AMESTICA LILLO ALVARO,43052,UNIDAD DROGUERÍA,1954,DEPTO. DE ABASTECIMIENTO DSSMO,ALVARO AMESTICA LILLO
24,CALFUMAN VALDEBENITO PATRICIA,1926,CENTRO INFANTIL,4042,DEPTO CALIDAD DE VIDA LABORAL DSSMO,PATRICIA CALFUMAN VALDEBENITO
25,SEPULVEDA GALDAMES ANGELICA,40351,UNIDAD ESTADISTICA Y GEST. ADM. PACIENTE,4035,DEPTO INFORMACION SANITARIA DSSMO,ANGELICA SEPULVEDA GALDAMES
27,TAPIA VILLA ELIANA,1926,CENTRO INFANTIL,4042,DEPTO CALIDAD DE VIDA LABORAL DSSMO,ELIANA TAPIA VILLA
34,SANCHEZ MORALES GEMITA,1910,DEPARTAMENTO DE FINANZAS,1900,SUBDIRECCION ADMINISTRATIVA DSSMO,GEMITA SANCHEZ MORALES
39,CORONA FERNANDEZ JULIO,1964,DEPTO DE GESTION DE LAS PERSONAS,4038,SUBDIR GESTION Y DESARROLL PERSONAS SSMO,JULIO CORONA FERNANDEZ
42,ARRIAGADA BECERRA ADRIANA,1910,DEPARTAMENTO DE FINANZAS,1900,SUBDIRECCION ADMINISTRATIVA DSSMO,ADRIANA ARRIAGADA BECERRA
